# Implementing the cropping function

## The various sub-tasks involved are :

* Get shape of the big image : (p,q)
* Define the size of the tile : (a,b) = (50,50)
* Determine the number of useful tiles; ensure all tiles are of equal size (a,b) and discard the ones that arent - Might lose a few due to boundary conditions but thats okay!
* Carry out Cropping of the tiles sequentially! 
* Obtain the vertex of the each of the cropped tiles.
* Save each of these cropped tiles in respective folder.

In [1]:
#Import Stuff 
import os
import cv2
import math
import numpy as np
import pandas as pd
import pickle

In [ ]:
#Read the image
#source_path = '/Users/keshavaprasad/Desktop/image/MG48_3day_bs/testingcrop/S051_00.tif'
#source_path = '/Users/keshavaprasad/Desktop/image/S25/S25_00.tif'
for x in range(1,100):
    print(x)
    source_path='//ibs9010/current_data/Data_Keshav/masked_for_rabies_auto_detection/MG48_3day_bs/Image/pia_only/S{}_Pia_Only.tif'.format(x)
    image=cv2.imread(source_path)
    
        # Image Basics
    print(image.shape)
    # Get Shape
    p = image.shape[0]# p
    q = image.shape[1]#q
    print("Height:",p,"Width:",q)

    # Defining the dimensions of a tile 
    #a = b = 50
    a = b = 70
    #************************
    cropped_tile_images = [] #list for storing all cropped tile images
    vertices = [] #list for storing lists of vertex values # Defining vertices of tiles for cropping
    ctc = []#list for storing lists of tile numbers
    # Cropping image Sequentially
    i = j = cropped_tile_counter = correct_tile_counter = wrong_dim_tile_counter = correct_dim_tile_counter= 0
    
    
    # Carry oout the sequential cropping store data into respective lists
    for i in range(0,math.floor(p),a):#range(start, stop, step) # to get the resolution right!
        for j in range(0,math.floor(q),b):
            #Defining the cropping box
            #tile = image[j:j+b,i:i+a]#y1: y2, x1: x2
            y1 = j
            x1 = i
            y2 = j+b
            x2 = i+a

            tile = image[x1:x2,y1:y2]# making crops of each subimage and then storing in tile variable and then appended to cropped_tiles list
            vertex = [x1,x2,y1,y2] # list of vertex values of each tiles
            cropped_tile_counter +=1

            # Calculating mean for each tile and
            #mean = np.mean(tile)

            # checking for tiles with wrong dimensions
            #if tile.shape[0]!=50 or tile.shape[1]!= 50:
            if tile.shape[0]!=70 or tile.shape[1]!= 70:
                wrong_dim_tile_counter += 1

            # Keeping only tiles with correct dimensions # Append the vertex for each of these tiles into vertices list 
            # Append the correct_tile_counter variable into ctc list

            #if tile.shape[0] == 50 and tile.shape[1] == 50:
            if tile.shape[0] == 70 and tile.shape[1] == 70 and not np.all((np.array(tile) == 0)):
                correct_dim_tile_counter +=1
                correct_tile_counter +=1
                #cropped_tiles.append(tile)
                cropped_tile_images.append(tile)
                #ctc.append([cropped_tile_counter,correct_tile_counter])
                ctc.append(correct_tile_counter)
                #print("correct_tile_counter:",correct_tile_counter,"with vertex:",vertex)
                vertices.append(vertex)  

        

    # Calling DataFrame constructor after zipping # both lists, with columns specified 
    cropped_data = pd.DataFrame(list(zip(ctc, vertices)),columns =['Tile_Counter', 'Vertices']) 
    #cropped_data = pd.DataFrame(list(zip(cropped_tiles,ctc, vertices)),columns =['Cropped_Tile_Counter','Correct_Tile_Counter', 'Vertices']) 
    #cropped_data cropped_data
    cropped_data

    
    pickle_path = '//ibs9010/current_data/Data_Keshav/image/Classification_model/'
    
    with open(pickle_path + 'global_xy_MG48_3day_bs_S{}_bigtile_piamasked.pkl'.format(x), 'wb') as g:
        pickle.dump(cropped_data, g)
        
        
    print("Total number of tiles:",cropped_tile_counter)
    print("Wasted tiles due boundary conditions:",(wrong_dim_tile_counter/cropped_tile_counter)*100,"%")
    print("Useless background tiles:",(correct_dim_tile_counter - correct_tile_counter)/cropped_tile_counter*100,"%")
    print("tiles with tissue:",(correct_tile_counter/cropped_tile_counter)*100,"%")
    
    # Saving the tiles in a respective folder
    #Dest_folder = "//ibs9010/current_data/Data_Keshaviimage/Classification_model/testingcrop/"
    Dest_Path = '//ibs9010/current_data/Data_Keshav/Cropped_Images//S{}/'.format(x)
    
    if not os.path.exists(Dest_Path):
       print("path doesn't exist. trying to make the folder.")
       os.makedirs(Dest_Path)

    #Dest_Path = '/Users/keshavaprasad/Desktop/image/S25/'
    # Saving the tiles in a respective folder    
    for k in range(1,len(cropped_tile_images)):
        #pass
        #print("K:",k)
        #cv2.imwrite(str(k)+ ".tif", tiles[k])
        cv2.imwrite(os.path.join(Dest_Path,str(k)+".tif"), cropped_tile_images[k])

1
(3882, 6494, 3)
Height: 3882 Width: 6494
Total number of tiles: 5208
Wasted tiles due boundary conditions: 2.8417818740399383 %
Useless background tiles: 0.0 %
tiles with tissue: 43.24116743471582 %
2
(3890, 6568, 3)
Height: 3890 Width: 6568
Total number of tiles: 5264
Wasted tiles due boundary conditions: 2.830547112462006 %
Useless background tiles: 0.0 %
tiles with tissue: 42.32522796352583 %
3
(3872, 6543, 3)
Height: 3872 Width: 6543
Total number of tiles: 5264
Wasted tiles due boundary conditions: 2.830547112462006 %
Useless background tiles: 0.0 %
tiles with tissue: 44.338905775075986 %
4
(3877, 6539, 3)
Height: 3877 Width: 6539
Total number of tiles: 5264
Wasted tiles due boundary conditions: 2.830547112462006 %
Useless background tiles: 0.0 %
tiles with tissue: 44.148936170212764 %


In [ ]:
import os
from os.path import expanduser

home = expanduser('~')

dl_path = home + '/Downloads/PDMB'

def main():
    if not os.path.exists(dl_path):
       print "path doesn't exist. trying to make"
       os.makedirs(dl_path)



 if __name__ == '__main__':
    main()


In [ ]:
print(len(cropped_tiles))

In [ ]:
cropped_data

In [ ]:
#cropped_data.loc[cropped_data['Tile_Counter'] == 21555] #searching rows based on specific values
#cropped_data.loc[cropped_data['mean_values'] > 15] 

In [ ]:
print(len(cropped_tiles))
#print(cropped_tiles[1000][0])

## Saving useful tiles that are sequentially cropped and are of correct dimensions and qualify mean value filter